<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>

# 3.0 서버 성능

이 노트북에서는 앞서 배운 최적화 기법을 구현하고 결과 모델을 보다 공식적인 방법으로 프로파일링합니다.

**[3.1 최적화의 영향 평가](#3.1-최적화의-영향-평가)**<br>
&nbsp; &nbsp; &nbsp; &nbsp; [3.1.1 연습: 모델 프로파일링](#3.1.1-연습:-모델-프로파일링)<br>
**[3.2 성능 변동 모니터링 및 대응](#3.2-성능-변동-모니터링-및-대응)**<br>
&nbsp; &nbsp; &nbsp; &nbsp; [3.2.1 Prometheus 지표 보기](#3.2.1-Prometheus-지표-보기)<br>
&nbsp; &nbsp; &nbsp; &nbsp; [3.2.2 지표 해석](#3.2.2-지표-해석)<br>

요청 패턴의 특성이 추론 기능에 미치는 영향뿐 아니라 구성 변경 사항의 영향도 분석할 것입니다. 고급 Triton 기능이 활성화되지 않은 TorchScript 기반 모델의 성능과 앞서 배운 주요 기능이 활성화된 TensorRT ONNX 모델의 성능을 비교하기 위한 체계적인 보고서를 생성할 것입니다. 

수업의 이전 부분에서 분석한 기본 지표(처리량 및 지연 시간)를 중점적으로 살펴보고, 솔루션의 지연 시간에 영향을 미치는 요인(예: 네트워크 통신)을 파악해 볼 것입니다.

마지막으로, 프로덕션에서 솔루션의 성능을 모니터링 및 관리하는 데 사용할 수 있는 도구와 그러한 도구를 사용해 자동 확장과 같은 고급 기능을 구현할 수 있는 방법을 살펴볼 것입니다.

# 3.1 최적화의 영향 평가
지금까지 사용해 온 성능 도구에는 화면에 결과를 표시할 뿐 아니라 데이터를 다음 위치에 표 형식으로 저장하는 추가 기능이 있습니다. 

<code>"./results/${MODEL_NAME}/results${RESULTS_ID}_${TIMESTAMP}.csv"</code>

다양한 최적화의 영향을 평가하기 위해 이전에 생성한 로그 파일을 활용해 보겠습니다.

## 3.1.1 연습: 모델 프로파일링
앞서 <code>bertQA-torchscript</code>와 <code>bertQA-onnx-trt-dynbatch</code>를 실행했으므로 해당 실행의 로그가 이미 저장되어 있을 것입니다. 해당 로그 폴더의 내용을 살펴보겠습니다. 성능 도구를 두 번 이상 실행했다면 타임 스탬프가 각기 다른 여러 로그 파일이 생성되었을 것입니다.

In [ ]:
!ls ./results/bertQA-torchscript/
!ls ./results/bertQA-onnx-trt-dynbatch

CSV 파일을 모두 다운로드하십시오(왼쪽 창에서 파일을 찾고 마우스 오른쪽 버튼을 클릭해 "다운로드"를 찾음). 실행 보고서를 생성하려면 아래 단계에 따라 <code>bertQA-onnx-trt-dynbatch</code>의 로그 파일을 가져오십시오.

<!-- - [이 스프레드시트](Triton%20Inference%20Server%20Performance%20Results.xlsx) 열기 -->
- <a href="https://docs.google.com/spreadsheets/d/1S8h0bWBBElHUoLd2SOvQPzZzRiQ55xjyqodm_9ireiw/edit#gid=1572240508">이 스프레드시트</a> 열기
- 파일 메뉴 "복사…"에서 복사
- 복사본 열기
- "원시 데이터" 탭에서 A1 셀 선택
- 파일 메뉴에서 "가져오기…" 선택
- "업로드"를 선택하고 파일 업로드
- "선택된 셀의 데이터 바꾸기"를 선택한 다음 "데이터 가져오기" 버튼 선택

위 단계를 완료하면 "지연 시간 구성 요소" 탭과 "지연 시간 대 처리량" 탭에 각각 다음 도표가 표시되어야 합니다. <br/>
<img width=600 src="images/ComponentsOfLatency1.png"/> <img width=600 src="images/LatencyVsThrughput1.png"/> <br/>

<code>bertQA-torchscript</code> 모델에 위 단계를 반복하십시오. (TorchScript 변형은 배치 8에서 실행되었다는 점을 기억하십시오.) <br>
어떻게 비교됩니까? 강사와 논의하십시오.

`bertQA-torchscript` 모델에 대한 분석 이미지도 <a href="images/torchscript_latency1.png">여기</a>와 <a href="images/torchscript_latency2.png">여기</a>에서 찾을 수 있습니다.

# 3.2 성능 변동 모니터링 및 대응

추론 서버의 성능을 이해하는 것은 초기 계획 단계에서 중요할 뿐 아니라 애플리케이션의 수명 전체에서도 마찬가지로 중요합니다. 서버 성능을 설명하는 지표를 캡처할 수 있는 능력은 문제에 대응하는 능력의 핵심일 뿐 아니라 자동 확장과 같은 고급 기능의 토대이기도 합니다.  아래 다이어그램은 Triton 배포 아키텍처를 간단하게 보여줍니다. Triton과 [Kubernetes](https://kubernetes.io/docs/home/) 등의 기술을 결합하여 데이터센터 내에서 증가하는 요구에 맞춰 자동으로 확장하거나 필요한 경우 과도한 워크로드를 클라우드로 보내는 구성을 비교적 쉽게 만들 수 있습니다. <br/>

<img width=700 src="images/DeploymentArchitecture.png"/>

## 3.2.1 Prometheus 지표 보기
Triton은 기본적으로 포트 8002에 대한 모니터링을 위한 [Prometheus](https://prometheus.io/) 성능 지표를 노출합니다. 여기에는 GPU 전력 사용량, GPU 메모리, 요청 개수, 지연 시간 측정값에 대한 지표가 포함됩니다.  개별 지표에 대한 추가 문서는 <a href="https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/metrics.html">Triton 지표 문서</a>에서 찾을 수 있습니다. 이제 성능 실행 전체에서 캡처된 지표를 쿼리해 보겠습니다.

In [ ]:
# Set the server hostname and check it - you should get a message that "Triton Server is ready!"
tritonServerHostName = "triton"
!./utilities/wait_for_triton_server.sh {tritonServerHostName}

In [ ]:
# Use a curl command to request the metrics
prometheus_url = tritonServerHostName + ":8002/metrics"
!curl -v {prometheus_url}

## 3.2.2 지표 해석
Prometheus 지표 출력은 각각 다음과 같은 형태로 제공되는 지표의 목록입니다.

```
# HELP <metric_name and description>
# TYPE <metric_name and type>
metric_name{gpu_uuid="GPU-xxxxxx",...} <data>
```

예를 들어, 추론 서버 모델에 2개의 모델이 포함된 경우 각 모델에 고유한 지표와 두 모델이 공유하는 GPU에 대한 보다 일반적인 기타 지표가 목록에 표시되어야 합니다.<br>

#### 개수 예
다음 예는 `bertQA-onnx-trt-dynbatch` 모델의 추론 수는 지금까지 10,105개이고, `bertQA-torchscript` 모델의 추론 수는 717개임을 나타냅니다.<br>결과가 어떻게 표시됩니까?
```
# HELP nv_inference_count Number of inferences performed
# TYPE nv_inference_count counter
nv_inference_count{gpu_uuid="GPU-640c6e00-43dd-9fae-9f9a-cb6af82df8e9",model="bertQA-onnx-trt-dynbatch",version="1"} 10105.000000
nv_inference_count{gpu_uuid="GPU-640c6e00-43dd-9fae-9f9a-cb6af82df8e9",model="bertQA-torchscript",version="1"} 717.000000
```

#### GPU 전력 예
다음 예는 현재 GPU 전력 사용량이 약 40와트임을 나타냅니다.<br>결과가 어떻게 표시됩니까?
```
# HELP nv_gpu_power_usage GPU power usage in watts
# TYPE nv_gpu_power_usage gauge
nv_gpu_power_usage{gpu_uuid="GPU-640c6e00-43dd-9fae-9f9a-cb6af82df8e9"} 39.958000
```

#### 결과가 무엇을 나타냅니까?

* 현재 활용률을 식별할 수 있습니까? 
* 0인 이유는 무엇입니까? 
* 사용 중인 메모리 양은 얼마입니까? 
* 서버에 대해 실행된 요청이 없음에도 GPU 메모리를 사용하고 있는 이유는 무엇이라고 생각하십니까? 

강사와 논의하십시오.

<h3 style="color:green;">축하합니다!</h3><br>
최적화를 성공적으로 구성했으며 모델을 프로파일링하는 방법을 배웠습니다.<br>

마지막 수업으로 넘어가 Triton 기능을 활용하는 맞춤형 애플리케이션을 구축하는 방법을 배우십시오.<br>
[4.0 모델 사용](040_UsingTheModel.ipynb)

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="헤더" style="width: 400px;"/> </a>